# Creating supercells with pymatgen

The [Pymatgen](https://pymatgen.org/) python library allows to setup
solid-state calculations using a flexible set of classes as well as an API
to an online data base of structures. Its `Structure` and `Lattice`
objects are directly supported by the DFTK `load_atoms` and `load_lattice`
functions, such that DFTK may be readily used to run calculation on systems
defined in pymatgen. Using the `pymatgen_structure` function a conversion
from DFTK to pymatgen structures is also possible. In the following we
use this to create a silicon supercell and find its LDA ground state
using direct minimisation. To run this example Julia's `PyCall` package
needs to be able to find an installation of `pymatgen`.

First we setup the silicon lattice in DFTK.

In [1]:
using DFTK

a = 10.263141334305942  # Lattice constant in Bohr
lattice = a / 2 .* [[0 1 1.]; [1 0 1.]; [1 1 0.]]
Si = ElementPsp(:Si, psp=load_psp("hgh/lda/Si-q4"))
atoms = [Si => [ones(3)/8, -ones(3)/8]];

Next we make a `[2, 2, 2]` supercell using pymatgen

In [2]:
pystruct = pymatgen_structure(lattice, atoms)
pystruct.make_supercell([2, 2, 2])
lattice = load_lattice(pystruct)
atoms = [Si => [s.frac_coords for s in pystruct.sites]];

Setup an LDA model and discretize using
a single k-point and a small `Ecut` of 5 Hartree.

In [3]:
model = model_LDA(lattice, atoms)
basis = PlaneWaveBasis(model; Ecut=5, kgrid=(1, 1, 1))

PlaneWaveBasis discretization:
    Ecut                 : 5.0 Ha
    fft_size             : (32, 32, 32), 32768 total points
    kgrid type           : Monkhorst-Pack
    kgrid                : [1, 1, 1]
    num. irred. kpoints  : 1

    Discretized Model(lda_xc_teter93, 3D):
        lattice (in Bohr)    : [0         , 10.2631   , 10.2631   ]
                               [10.2631   , 0         , 10.2631   ]
                               [10.2631   , 10.2631   , 0         ]
        unit cell volume     : 2162.1 Bohr³
    
        atoms                : Si₁₆
        atom potentials      : ElementPsp(Si, psp="hgh/lda/si-q4")
    
        num. electrons       : 64
        spin polarization    : none
        temperature          : 0 Ha
    
        terms                : Kinetic()
                               AtomicLocal()
                               AtomicNonlocal()
                               Ewald()
                               PspCorrection()
                               

Find the ground state using direct minimisation (always using SCF is boring ...)

In [4]:
scfres = direct_minimization(basis, tol=1e-5);

Iter     Function value   Gradient norm 
     0     1.120028e+02     1.640208e+00
 * time: 0.40019822120666504
     1     1.092183e+01     8.736691e-01
 * time: 1.5821130275726318
     2    -1.137790e+01     1.022288e+00
 * time: 1.6468992233276367
     3    -3.415220e+01     7.119952e-01
 * time: 1.7700080871582031
     4    -4.755048e+01     5.475081e-01
 * time: 1.844135046005249
     5    -5.692776e+01     1.858093e-01
 * time: 1.9329001903533936
     6    -5.978803e+01     1.157083e-01
 * time: 1.9990651607513428
     7    -6.084729e+01     5.542336e-02
 * time: 2.075888156890869
     8    -6.126533e+01     7.332236e-02
 * time: 2.1308541297912598
     9    -6.161498e+01     3.196271e-02
 * time: 2.1859350204467773
    10    -6.184304e+01     2.643906e-02
 * time: 2.243234157562256
    11    -6.200240e+01     1.802944e-02
 * time: 2.2990081310272217
    12    -6.206678e+01     1.611707e-02
 * time: 2.373042106628418
    13    -6.212709e+01     1.390636e-02
 * time: 2.4283761978149

In [5]:
scfres.energies

Energy breakdown (in Ha):
    Kinetic             25.7671069
    AtomicLocal         -18.8557691
    AtomicNonlocal      14.8522656
    Ewald               -67.1831486
    PspCorrection       -2.3569765
    Hartree             4.8485375 
    Xc                  -19.3336821

    total               -62.261666461667